In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np

import scipy.optimize as optimization
from scipy.optimize import least_squares

In [2]:
#C:\ti\simplelink_cc13x2_26x2_sdk_4_40_04_04\tools\ble5stack\rtls_agent\examples\3ANT\ofoghi\rtls_aoa_iq_with_rtls_util_export_into_csv_3ant_AoA_log\04_16_2021_16_55_55_rtls_raw_iq_samples_0x11_zero_f88a5e2d7808_0.csv
# path = 'C:/ti/simplelink_cc13x2_26x2_sdk_4_40_04_04/tools/ble5stack/rtls_agent/examples/3ANT/ofoghi/rtls_aoa_iq_with_rtls_util_export_into_csv_3ant_AoA_log/04_16_2021_16_55_55_rtls_raw_iq_samples_0x11_zero_f88a5e2d7808_0.csv' # test @ 1m 
# path = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero_degree_zero_posi/05_18_2021_14_16_43_rtls_raw_iq_samples_f88a5e2d7808_0.csv'
# path = 'C:/ti/simplelink_cc13x2_26x2_sdk_4_40_04_04/tools/ble5stack/rtls_agent/examples/3ANT/ofoghi/rtls_aoa_iq_with_rtls_util_export_into_csv_3ant_AoA_log/05_19_2021_12_52_25_rtls_raw_iq_samples_0x10_30degreef88a5e2d7808_0.csv'
path =  'C:/ti/simplelink_cc13x2_26x2_sdk_4_40_04_04/tools/ble5stack/rtls_agent/examples/3ANT/ofoghi/rtls_aoa_iq_with_rtls_util_export_into_csv_3ant_AoA_log/05_25_2021_14_52_24_rtls_raw_iq_samples_0x11_f88a5e2d7808_0.csv' # for zero degree


data = pd.read_csv(path)
data_size = len(data)
N = int(data_size/624)
print('Number of pakts are', N, int(N/2), len(data))
t = np.arange(0, 156*N, 0.25)
print(len(t))
data['time_stamp'] = t
data['Magnitude'] = np.sqrt(np.power(data['i'], 2) + np.power(data['q'], 2))
print(len(np.unique(data.channel)))

Number of pakts are 69 34 43056
43056
33


In [3]:
data['ts'] = data['time_stamp']
t = data[data.channel >= 11]
t['omega'] = (t['channel']-11)*2+ 2428.25
tless = data[data.channel < 11]
tless['omega'] = (tless['channel'])*2 + 2404.25
data = t.append(tless)

<ipython-input-3-23b68b2b1de6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['omega'] = (t['channel']-11)*2+ 2428.25
<ipython-input-3-23b68b2b1de6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tless['omega'] = (tless['channel'])*2 + 2404.25


In [4]:
w = []
for n in range(0,11):
    w.append(2404.25 + n*2)
for n in range(11, 37):
    w.append(2428.25 + (n-11)*2)
w = np.asarray(w)
w*1e6

array([2.40425e+09, 2.40625e+09, 2.40825e+09, 2.41025e+09, 2.41225e+09,
       2.41425e+09, 2.41625e+09, 2.41825e+09, 2.42025e+09, 2.42225e+09,
       2.42425e+09, 2.42825e+09, 2.43025e+09, 2.43225e+09, 2.43425e+09,
       2.43625e+09, 2.43825e+09, 2.44025e+09, 2.44225e+09, 2.44425e+09,
       2.44625e+09, 2.44825e+09, 2.45025e+09, 2.45225e+09, 2.45425e+09,
       2.45625e+09, 2.45825e+09, 2.46025e+09, 2.46225e+09, 2.46425e+09,
       2.46625e+09, 2.46825e+09, 2.47025e+09, 2.47225e+09, 2.47425e+09,
       2.47625e+09, 2.47825e+09])

In [7]:
import ctypes
I = data['i']
Q = data['q']
I_lst = []
Q_lst = []

for i, q in zip(I, Q):
    I_lst.append(ctypes.c_byte(i).value)
    Q_lst.append(ctypes.c_byte(q).value)
data['i_byt'] = I_lst
data['q_byt'] = Q_lst

In [8]:
data['Mag_new'] = np.sqrt(np.power(data['i_byt'], 2) + np.power(data['q_byt'], 2))

### NOW Optimize the RAW_IQ samples

In [9]:
# optomize the data from ANT1
def s(theta, t ):
    x = theta[0] * np.cos(2*np.pi*t*fc + phi)
    y = theta[1] * np.sin(2*np.pi*t*fc + phi)
    return np.array([x, y])

new_i_lst = []
new_q_lst = []

phi = 0;
for i in range(0, len(data), 624):
    Ai = data.Magnitude[i:i+624].mean()
    Aq = data.Magnitude[i:i+624].mean()
    fc = data.omega[i:i+624].mean(); 
    ts = data.time_stamp[i:i+624]
    ss = s([Ai, Aq, fc, phi], ts)
    ss[0] += data['i_byt'][i:i+624]
    ss[1] += data['q_byt'][i:i+624]
    

    def fun(theta):
        return (ss- s(theta, ts)).flatten()

    theta0 = [0,0,0,0]
    res1 = least_squares(fun, theta0)
    N = int(len(res1.fun)/2)
    new_i1 = res1.fun[0:N]
    new_i_lst.append(new_i1)
    new_q1 = res1.fun[N:len(res1.fun)]
    new_q_lst.append(new_q1)

In [169]:
# plt.scatter(data['i'][0:624], data['q'][0:624])
# plt.scatter(new_i_lst[0], new_q_lst[0])

In [10]:
from itertools import chain
flatten_i = list(chain.from_iterable(new_i_lst))
flatten_q = list(chain.from_iterable(new_q_lst))
data['new_i'] = flatten_i
data['new_q'] = flatten_q

In [11]:
info =[]
data_size = len(data)
N = int(data_size/624)
for n in range(N):
    for i in range(36+n*624, 624*(n+1), 8):
        for j in range(i, i+4):
            info.append({"pkt": n, "Channel": data['channel'][j], "sample_idx": data['sample_idx'][j], 
                         "I": data['i'][j],
                         "Q":data['q'][j],
                         "Magnitude": data['Magnitude'][j],
                         "time_stamp": data['time_stamp'][j], "omega": data['omega'][j], 
                         "new_i": data['new_i'][j], "new_q": data['new_q'][j], 
                         "I_8bit": data['i_byt'][j], "Q_8bit": data['q_byt'][j],
                         "Mag_new": data['Mag_new'][j]})
    info_dec = pd.DataFrame(info)

In [12]:
df = info_dec.drop(info_dec[info_dec.sample_idx > 607].index)

In [13]:
lst_ant1 = []
lst_ant2 = []
lst_ant3 = []

for i in range(0, len(df), 12):    
    lst_ant2.append(df[i:i+4])
    lst_ant1.append(df[i+4:i+8])
    lst_ant3.append(df[i+8:i+12])

In [14]:
df_ant1 = pd.concat(lst_ant1,  ignore_index=True)
df_ant2 = pd.concat(lst_ant2,  ignore_index=True)
df_ant3 = pd.concat(lst_ant3,  ignore_index=True)

### Caluclate the phase difference 

In [17]:
X12 = (df_ant1['I'] + 1j*df_ant1['Q']) * (df_ant2['I'] - 1j*df_ant2['Q'])
X23 = (df_ant2['I'] + 1j*df_ant2['Q']) * (df_ant3['I'] - 1j*df_ant3['Q'])
X13 = (df_ant1['I'] + 1j*df_ant1['Q']) * (df_ant3['I'] - 1j*df_ant3['Q'])
X12_opt = (df_ant1['new_i'] + 1j*df_ant1['new_q']) * (df_ant2['new_i'] - 1j*df_ant2['new_q'])
X23_opt = (df_ant2['new_i'] + 1j*df_ant2['new_q']) * (df_ant3['new_i'] - 1j*df_ant3['new_q'])
X13_opt = (df_ant1['new_i'] + 1j*df_ant1['new_q']) * (df_ant3['new_i'] - 1j*df_ant3['new_q'])
X12_8bit = (df_ant1['I_8bit'] + 1j*df_ant1['Q_8bit']) * (df_ant2['I_8bit'] - 1j*df_ant2['Q_8bit'])
X23_8bit = (df_ant2['I_8bit'] + 1j*df_ant2['Q_8bit']) * (df_ant3['I_8bit'] - 1j*df_ant3['Q_8bit'])
X13_8bit = (df_ant1['I_8bit'] + 1j*df_ant1['Q_8bit']) * (df_ant3['I_8bit'] - 1j*df_ant3['Q_8bit'])

In [18]:
d = {'channel': df_ant1['Channel'],'omega':df_ant1['omega']}
data_fin = pd.DataFrame(data=d)
data_fin['phi12'] = np.angle(X12)
data_fin['phi23'] = np.angle(X23)
data_fin['phi13'] = np.angle(X13)
data_fin['mag12'] = np.abs(X12)
data_fin['mag23'] = np.abs(X23)
data_fin['mag13'] = np.abs(X13)

In [19]:
data_fin['phi12_opt'] = np.angle(X12_opt)
data_fin['phi23_opt'] = np.angle(X23_opt)
data_fin['phi13_opt'] = np.angle(X13_opt)
data_fin['mag12_opt'] = np.abs(X12_opt)
data_fin['mag23_opt'] = np.abs(X23_opt)
data_fin['mag13_opt'] = np.abs(X13_opt)

In [20]:
data_fin['phi12_8bit'] = np.angle(X12_8bit)
data_fin['phi23_8bit'] = np.angle(X23_8bit)
data_fin['phi13_8bit'] = np.angle(X13_8bit)
data_fin['mag12_8bit'] = np.abs(X12_8bit)
data_fin['mag23_8bit'] = np.abs(X23_8bit)
data_fin['mag13_8bit'] = np.abs(X13_8bit)

In [21]:
real_phi= 0
cos = np.cos(np.deg2rad(real_phi))
landa = 2.9e8/(data_fin.omega*1e6)
d12 = 3.5e-2
# ex_phi12 = (cos45* 2*np.pi*d12)/landa
# ex_phi13 = (cos45*2*2*np.pi*d12)/landa
ex_phi12 = (cos* 2*np.pi*d12)/landa
ex_phi13 = (cos*2*2*np.pi*d12)/landa

In [163]:
# data_fin['phi12_exp'] = data_fin['phi12'] - ex_phi12
# data_fin['phi23_exp'] = data_fin['phi23'] - ex_phi12
# data_fin['phi13_exp'] = data_fin['phi13'] - ex_phi13
# data_fin['phi12_exp_opt'] = data_fin['phi12_opt'] - ex_phi12
# data_fin['phi23_exp_opt'] =  data_fin['phi23_opt'] - ex_phi12
# data_fin['phi13_exp_opt'] =  data_fin['phi13_opt'] - ex_phi13
# matlab_path = 'C:/Users/pooneh/Documents/MATLAB/AoA'
# data_fin.to_csv(matlab_path+'_all_points_ref.csv', index = False)

In [22]:
phi_channel = []
for i in range(0, len(data_fin),96):
    phi_channel.append({"channel":data_fin['channel'][i], "omega":data_fin['omega'][i],
                       "ave_phi12": np.average(data_fin['phi12'][i:i+96]),
                       "ave_phi23": np.average(data_fin['phi23'][i:i+96]),
                        "ave_phi13": np.average(data_fin['phi13'][i:i+96]),
                        "ave_phi12_opt": np.average(data_fin['phi12_opt'][i:i+96]),
                       "ave_phi23_opt": np.average(data_fin['phi23_opt'][i:i+96]),
                        "ave_phi13_opt": np.average(data_fin['phi13_opt'][i:i+96]),
                       "ave_mag12": np.average(data_fin['mag12'][i:i+96]),
                       "ave_mag23": np.average(data_fin['mag23'][i:i+96]),
                       "ave_mag13": np.average(data_fin['mag13'][i:i+96]),
                       "ave_phi12_8bit": np.average(data_fin['phi12_8bit'][i:i+96]),
                       "ave_phi23_8bit": np.average(data_fin['phi23_8bit'][i:i+96]),
                        "ave_phi13_8bit": np.average(data_fin['phi13_8bit'][i:i+96]),})
average_perchannel = pd.DataFrame(phi_channel)

In [23]:
ave_phi12 = np.average(average_perchannel['ave_phi12'])*180/np.pi
ave_phi23 = np.average(average_perchannel['ave_phi23'])*180/np.pi,
ave_phi13 = np.average(average_perchannel['ave_phi13'])*180/np.pi
total_ave = (ave_phi12+ ave_phi23 + ave_phi13)/3

In [24]:
ave_phi12_opt = np.average(average_perchannel['ave_phi12_opt'])*180/np.pi
ave_phi23_opt = np.average(average_perchannel['ave_phi23_opt'])*180/np.pi,
ave_phi13_opt = np.average(average_perchannel['ave_phi13_opt'])*180/np.pi
total_ave_opt = (ave_phi12_opt+ ave_phi23_opt + ave_phi13_opt)/3

In [25]:
ave_phi12_8bit = np.average(average_perchannel['ave_phi12_8bit'])*180/np.pi
ave_phi23_8bit = np.average(average_perchannel['ave_phi23_8bit'])*180/np.pi,
ave_phi13_8bit = np.average(average_perchannel['ave_phi13_8bit'])*180/np.pi
total_ave_8bit = (ave_phi12_8bit+ ave_phi23_8bit + ave_phi13_8bit)/3

In [26]:
print(' Angle per paire_8bit', ave_phi12_8bit, ave_phi23_8bit, ave_phi13_8bit, '\n Total average of angle_8bit', total_ave_8bit)

 Angle per paire_8bit 5.97187655927672 (-0.5184573974949046,) -8.62266779473993 
 Total average of angle_8bit [-1.05641621]


In [27]:
ave_phi12_opt_rad = np.average(average_perchannel['ave_phi12_opt'])
ave_phi23_opt_rad = np.average(average_perchannel['ave_phi23_opt'])
ave_phi13_opt_rad = np.average(average_perchannel['ave_phi13_opt'])
total_ave_opt_rad = (ave_phi12_opt_rad+ ave_phi23_opt_rad + ave_phi13_opt_rad)/3

In [28]:
print(' Angle per paire', ave_phi12, ave_phi23, ave_phi13, '\n Total average of angle', total_ave)

 Angle per paire 87.50993495624061 (3.5994935035509275,) -90.08622371412152 
 Total average of angle [0.34106825]


In [29]:
print(' Angle per paire', ave_phi12_opt_rad, ave_phi23_opt_rad, ave_phi13_opt_rad, '\n Total average of angle', total_ave_opt_rad)

 Angle per paire 0.10236810971362613 -0.01657546647709377 -0.15513867741920662 
 Total average of angle -0.02311534472755809


In [30]:
print(' Angle per paire OPTIMIZED', ave_phi12_opt, ave_phi23_opt, ave_phi13_opt, '\n Total average of angle_opt', total_ave_opt)

 Angle per paire OPTIMIZED 5.8652606433229435 (-0.9497042725980521,) -8.888791455362066 
 Total average of angle_opt [-1.32441169]


In [33]:
real_phi= 90
cos = np.cos(np.deg2rad(real_phi))
landa = 2.9e8/(average_perchannel.omega*1e6)
d12 = 3.5e-2
# ex_phi12 = (cos45* 2*np.pi*d12)/landa
# ex_phi13 = (cos45*2*2*np.pi*d12)/landa
ex_phi12 = (cos* 2*np.pi*d12)/landa
ex_phi13 = (cos*2*2*np.pi*d12)/landa
ex_phi12.mean()

1.133858949732564e-16

In [34]:
landa_w = 2.9e8/(w*1e6)
ex_phi12_w = (cos* 2*np.pi*d12)/landa_w
ex_phi12_w.mean()

1.13374436664898e-16

In [35]:
ex_phi12.mean()

1.133858949732564e-16

In [36]:
np.arccos(ex_phi12* landa/(2*np.pi*d12))*180/np.pi

0     90.0
1     90.0
2     90.0
3     90.0
4     90.0
      ... 
64    90.0
65    90.0
66    90.0
67    90.0
68    90.0
Name: omega, Length: 69, dtype: float64

In [37]:
average_perchannel['ex_phi12'] = ex_phi12
average_perchannel['ex_phi13'] = ex_phi13

In [38]:
correction_coef = (ave_phi12_opt_rad - ex_phi12)

In [39]:
average_perchannel['correction_phi12'] = average_perchannel.ave_phi12 - average_perchannel.ex_phi12
average_perchannel['correction_phi13'] = average_perchannel.ave_phi13 - average_perchannel.ex_phi13
average_perchannel['correction_phi23'] = average_perchannel.ave_phi23  - average_perchannel.ex_phi12

In [43]:
average_perchannel['correction_phi12_8bit'] = average_perchannel.ave_phi12_8bit - average_perchannel.ex_phi12
average_perchannel['correction_phi13_8bit'] = average_perchannel.ave_phi13_8bit - average_perchannel.ex_phi13
average_perchannel['correction_phi23_8bit'] = average_perchannel.ave_phi23_8bit  - average_perchannel.ex_phi12

In [23]:
ave_phi12 = np.average(average_perchannel['ave_phi12'])*180/np.pi
ave_phi23 = np.average(average_perchannel['ave_phi23'])*180/np.pi,
ave_phi13 = np.average(average_perchannel['ave_phi13'])*180/np.pi
total_ave = (ave_phi12+ ave_phi23 + ave_phi13)/3

In [40]:
ave_phi12_corr = np.average(average_perchannel['correction_phi12'])*180/np.pi
ave_phi23_corr = np.average(average_perchannel['correction_phi13'])*180/np.pi,
ave_phi13_corr = np.average(average_perchannel['correction_phi23'])*180/np.pi
total_ave_corr = (ave_phi12_corr+ ave_phi23_corr + ave_phi13_corr)/3

In [41]:
print(' Angle per paire_corrected', ave_phi12_corr, ave_phi23_corr, ave_phi13_corr, '\n Total average of angle_corrected', total_ave_corr)

 Angle per paire_corrected 87.50993495624061 (-90.08622371412153,) 3.5994935035509212 
 Total average of angle_corrected [0.34106825]


In [42]:
np.arccos(ave_phi12_corr/(180/np.pi)*landa/(2*np.pi*d12))*180/np.pi

0     34.848562
1     34.373629
2     33.098589
3     34.578537
4     33.316759
        ...    
64    33.957483
65    35.048745
66    34.166625
67    35.246968
68    33.746156
Name: omega, Length: 69, dtype: float64

### Creating the correcting__file 

In [44]:
d2 = {"phi12_correction": average_perchannel.correction_phi12, 
     "phi23_correction": average_perchannel.correction_phi23,
     "phi13_correction": average_perchannel.correction_phi13,
     "phi12_correction_8bit": average_perchannel.correction_phi12_8bit, 
     "phi23_correction_8bit": average_perchannel.correction_phi23_8bit,
     "phi13_correction_8bit": average_perchannel.correction_phi13_8bit,
     "channel":average_perchannel.channel,
     "omega":average_perchannel.omega,
     "ex_phi12": ex_phi12, 
     "ave_phi12":average_perchannel.ave_phi12,
     "ave_phi23":average_perchannel.ave_phi23,
     "ave_phi13":average_perchannel.ave_phi13,
     "ave_phi12_8bit":average_perchannel.ave_phi12_8bit,
     "ave_phi23_8bit":average_perchannel.ave_phi23_8bit,
     "ave_phi13_8bit":average_perchannel.ave_phi13_8bit,}
correction_coeff = pd.DataFrame(d2)
path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero_degree_zero_posi'
correction_coeff.to_csv(path_coeff+'_correction_coeff_8bit.csv', index = False)

In [45]:
correction_coeff[35:72]

,phi12_correction,phi23_correction,phi13_correction,phi12_correction_8bit,phi23_correction_8bit,phi13_correction_8bit,channel,omega,ex_phi12,ave_phi12,ave_phi23,ave_phi13,ave_phi12_8bit,ave_phi23_8bit,ave_phi13_8bit
35,2.899201,0.338457,-2.849177,-1.793328,-0.938693,1.980368,32,2470.25,1.147022e-16,2.899201,0.338457,-2.849177,-1.793328,-0.938693,1.980368
36,1.336732,-0.094219,-1.375481,-0.079614,0.383607,0.042194,7,2418.25,1.122876e-16,1.336732,-0.094219,-1.375481,-0.079614,0.383607,0.042194
37,2.690107,0.268577,-2.931802,-1.106578,-0.120830,1.259686,16,2438.25,1.132163e-16,2.690107,0.268577,-2.931802,-1.106578,-0.120830,1.259686
38,2.943373,0.404272,-2.935540,0.450289,0.010519,-0.390041,0,2404.25,1.116376e-16,2.943373,0.404272,-2.935540,0.450289,0.010519,-0.390041
39,2.968071,0.361944,-2.953170,0.119475,-0.253299,-0.264724,21,2448.25,1.136806e-16,2.968071,0.361944,-2.953170,0.119475,-0.253299,-0.264724
40,2.392328,0.303484,-2.474727,2.392328,0.303484,-2.474727,5,2414.25,1.121019e-16,2.392328,0.303484,-2.474727,2.392328,0.303484,-2.474727
41,1.896630,0.064152,-2.031659,-0.005120,-0.306221,-0.114992,26,2458.25,1.141450e-16,1.896630,0.064152,-2.031659,-0.005120,-0.306221,-0.114992
42,1.271514,-0.678546,-1.828677,-0.139032,-0.436106,-0.051539,23,2452.25,1.138664e-16,1.271514,-0.678546,-1.828677,-0.139032,-0.436106,-0.051539
43,2.208906,0.222564,-2.542719,-0.423439,0.225560,0.522069,10,2424.25,1.125662e-16,2.208906,0.222564,-2.542719,-0.423439,0.225560,0.522069
44,1.662580,-0.367273,-1.519036,-0.792312,0.111438,0.562673,31,2468.25,1.146093e-16,1.662580,-0.367273,-1.519036,-0.792312,0.111438,0.562673
